In [1]:
import numpy as np
import os

log_dir = 'pi_log'
log_list = []
time_list = []
for _, _, files in os.walk(log_dir):
    for f in files:
        fname = os.path.join(log_dir, f)
        if fname.endswith('.log'):
            log_list.append(fname)
        else:
            time_list.append(fname)
# print(time_list)
# print(log_list)

In [2]:
import re

def parse_time(l):
    pattern = '[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{6}'
    base = os.path.basename(l)
    return re.search(pattern, base).group()

print(log_list[0])
t = parse_time(log_list[0])

print(t)

pi_log/log_2021-07-15T011027.log
2021-07-15T011027


In [3]:
def parse_block(block, name, host):
    m = {}
    m['func'] = block[0]
    meter_num = 0
    meter_time = 0
    control =['server', 'supplier', 'center']
    ta = ['ttp', 'kgc', 'ta']
    m['control'] = 0
    m['ta'] = 0
    m['agg'] = 0
    for r in block[2::2]:
        x, y = r.split(':')
        t = sum(int(e) for e in y.split(','))
        if x.startswith('meter_'):
            meter_num += 1
            meter_time += t
        elif x in control:
            m['control'] = +t
        elif x in ta:
            m['ta'] = t
        else:
            m[x] = t
    
    m['meter_num'] = meter_num
    m['meter_time'] = meter_time
    m['name'] = name
    m['host'] = host
    return m

def parse_record(lines):
    pattern = '[^ ]*$'
    host_pattern = '[^ ]*'
    name_pattern = 'bjut\.(.*)\.myMain'
    host = re.search(host_pattern, lines[0]).group()
    name = re.findall(name_pattern, lines[0])[0]

    func_block = []
    block = []
    for line in lines:
        s = re.search(pattern, line).group().strip()
        # print(s)
        # parse function block
        if ':' not in s: # enter function
            if len(block) != 0:
                func_block.append(block)
            block = []
            block.append(s)
        else:
            block.append(s)
    if len(block) != 0:
        func_block.append(block)

    # print(func_block[0])
    record_list = []
    for b in func_block:
        r = parse_block(b, name, host)
        record_list.append(r)
    if name == 'cir':
        for r in record_list[2:15:3]:
            r['func'] = r['func'] + '_3'
        k = 2
        for r in record_list[17:30:3]:
            r['func'] = r['func'] + '_' + str(k)
            k += 1
        k = 2
        for r in record_list[30:]:
            r['func'] = r['func'] + '_r' + str(k)
            k += 1
        record_list = record_list[1:]
    return record_list

In [4]:


import csv

fields_name = []
with open('data.csv', 'w', newline='') as csvfile:
    writer = None
    for l in log_list:
        log_time = parse_time(l)
        with open(l) as fo:
            record = parse_record(fo.readlines())
            if len(fields_name) == 0:
                fields_name  = fields_name + [k for k in record[0].keys()]
                fields_name.append('timestamp')
                writer = csv.DictWriter(csvfile, fieldnames=fields_name)
                writer.writeheader()
                print(fields_name)
            for l in record:
                l['timestamp'] = log_time
                writer.writerow(l)

['func', 'control', 'ta', 'agg', 'meter_num', 'meter_time', 'name', 'host', 'timestamp']
